In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from pyoscomp.scenario.components import (
    TopologyComponent,
    TimeComponent,
    DemandComponent,
    SupplyComponent,
    PerformanceComponent,
    EconomicsComponent
)

In [3]:
scenario_dir = 'demo_output/scenario1'
os.makedirs(scenario_dir, exist_ok=True)
setup_dir = os.path.join(scenario_dir, 'setup')
results_dir = os.path.join(scenario_dir, 'results')
os.makedirs(setup_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Use otoole to initialize scenario structure
!otoole setup csv demo_output/scenario1/setup --overwrite

In [4]:
topology = TopologyComponent(setup_dir)
topology.add_nodes(['REGION1'])
topology.save()

In [5]:
time = TimeComponent(setup_dir)
time.add_time_structure(
    years=[2026],
    seasons={'ALLSEASONS': 365},
    daytypes={'ALLDAYS': 1},
    brackets={'ALLTIMES': 24},
)
time.save()

In [6]:
demand = DemandComponent(setup_dir)
demand.add_annual_demand('REGION1', 'ELEC', {2026: 100})
demand.process()
demand.save()

In [7]:
supply = SupplyComponent(setup_dir)

supply.add_technology('REGION1', 'GAS_CCGT') \
    .with_operational_life(30) \
    .with_residual_capacity(0) \
    .as_conversion(input_fuel='GAS', output_fuel='ELEC')

supply.add_technology('REGION1', 'GAS_TURBINE') \
    .with_operational_life(25) \
    .with_residual_capacity(0) \
    .as_conversion(input_fuel='GAS', output_fuel='ELEC')

supply.save()

In [8]:
performance = PerformanceComponent(setup_dir)

performance.set_efficiency('REGION1', 'GAS_CCGT', 0.5)
performance.set_capacity_factor('REGION1', 'GAS_CCGT', 0.9)
performance.set_availability_factor('REGION1', 'GAS_CCGT', 1.0)
performance.set_capacity_to_activity_unit('REGION1', 'GAS_CCGT', 8760)
performance.set_capacity_limits('REGION1', 'GAS_CCGT',
    max_capacity={2026: 1000 / 8760}, min_capacity=0)

performance.set_efficiency('REGION1', 'GAS_TURBINE', 0.4)
performance.set_capacity_factor('REGION1', 'GAS_TURBINE', 0.8)
performance.set_availability_factor('REGION1', 'GAS_TURBINE', 1.0)
performance.set_capacity_to_activity_unit('REGION1', 'GAS_TURBINE', 8760)
performance.set_capacity_limits('REGION1', 'GAS_TURBINE',
    max_capacity={2026: 1000 / 8760}, min_capacity=0)

performance.process()
performance.save()


In [9]:
economics = EconomicsComponent(setup_dir)
economics.set_discount_rate('REGION1', 0.05)
economics.set_capital_cost('REGION1', 'GAS_CCGT', 500)
economics.set_variable_cost('REGION1', 'GAS_CCGT','MODE1', 2)
economics.set_fixed_cost('REGION1', 'GAS_CCGT', 0)
economics.set_capital_cost('REGION1', 'GAS_TURBINE', 400)
economics.set_variable_cost('REGION1', 'GAS_TURBINE', 'MODE1', 5)
economics.set_fixed_cost('REGION1', 'GAS_TURBINE', 0)
economics.save()

In [10]:
from pyoscomp.interfaces import ScenarioData

data = ScenarioData.from_directory(setup_dir, validate=True)
# Alternatively: 
data = ScenarioData.from_components(
    topology,
    time,
    demand,
    supply,
    performance,
    economics, validate=True)

In [11]:
# %%capture
# from pyoscomp.runners.osemosys import OSeMOSYSRunner

# osm = OSeMOSYSRunner(scenario_dir=scenario_dir,
#                      use_otoole=True)
# osm.run()

In [12]:
!otoole convert csv datafile demo_output/scenario1/setup demo_output/scenario1/scenario1.txt ../pyoscomp/OSeMOSYS_config.yaml

In [13]:
from IPython.utils.capture import capture_output

with capture_output() as cap:
    !glpsol -m ../pyoscomp/OSeMOSYS.txt -d demo_output/scenario1/scenario1.txt --wglp demo_output/scenario1/scenario1.glp --write demo_output/scenario1/scenario1.sol

# Show output only if not successful
expected_output = "model has been successfully processed"
if expected_output not in cap.stdout.lower():
    cap.show()

In [14]:
!otoole results glpk csv demo_output/scenario1/scenario1.sol demo_output/scenario1/results datafile demo_output/scenario1/scenario1.txt ../pyoscomp/OSeMOSYS_config.yaml --glpk_model demo_output/scenario1/scenario1.glp

In [15]:
import pandas as pd

# Load and Display OSeMOSYS Results
print("\n--- OSeMOSYS Optimization Results ---")
osemosys_objective = pd.read_csv(os.path.join(results_dir, 'TotalDiscountedCost.csv'))
print("\nObjective:", osemosys_objective)
osemosys_total_capacity = pd.read_csv(os.path.join(results_dir, 'TotalCapacityAnnual.csv'))
print("\nOptimal Capacities (p_nom_opt):\n", osemosys_total_capacity)
osemosys_total_production = pd.read_csv(os.path.join(results_dir, 'TotalTechnologyAnnualActivity.csv'))
print("\nTotal Production:\n", osemosys_total_production)


--- OSeMOSYS Optimization Results ---

Objective:     REGION  YEAR       VALUE
0  REGION1  2026  195.572923

Optimal Capacities (p_nom_opt):
     REGION   TECHNOLOGY  YEAR     VALUE
0  REGION1     GAS_CCGT  2026  0.012684
1  REGION1  GAS_TURBINE  2026  0.000000

Total Production:
     REGION   TECHNOLOGY  YEAR  VALUE
0  REGION1     GAS_CCGT  2026  100.0
1  REGION1  GAS_TURBINE  2026    0.0


In [16]:
from pyoscomp.translation.pypsa_translator import PyPSAInputTranslator

# --- Path B: PyPSA ---
translator = PyPSAInputTranslator(data)
network = translator.translate()

INFO:pypsa.network.index:Repeating time-series for each investment period and converting snapshots to a pandas.MultiIndex.


In [17]:
# Run PyPSA optimization
network.optimize(
    solver_name='glpk',
    multi_investment_periods=True
)

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.solvers:GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /var/folders/ny/csqtllsd23b8m43n1kthbfhm0000gp/T/linopy-problem-nzz2qs4j.lp
 --output /var/folders/ny/csqtllsd23b8m43n1kthbfhm0000gp/T/linopy-solve-7gyms_0s.sol
Reading problem data from '/var/folders/ny/csqtllsd23b8m43n1kthbfhm0000gp/T/linopy-problem-nzz2qs4j.lp'...
9 rows, 4 columns, 12 non-zeros
55 lines were read
GLPK Simplex Optimizer 5.0
9 rows, 4 columns, 12 non-zeros
Preprocessing...
3 rows, 4 columns, 6 non-zeros
Scaling...
 A: min|aij| =  8.000e-01  max|aij| =  1.000e+00  ratio =  1.250e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
      0: obj =   5.000000000e+02 inf =   1.142e-02 (1)
      2: obj =   5.004049798e+02 inf =   0.000e+00 (0)
*     4: obj =   2.004125535e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 sec

('ok', 'optimal')

In [18]:
from pyoscomp.translation.pypsa_translator import PyPSAOutputTranslator
from pyoscomp.translation.osemosys_translator import OSeMOSYSOutputTranslator
from pyoscomp.interfaces.results import compare

# --- PyPSA results → ModelResults ---
pypsa_results = PyPSAOutputTranslator(network).translate()
print(pypsa_results)
print()

# --- OSeMOSYS results → ModelResults ---
osemosys_results = OSeMOSYSOutputTranslator(results_dir).translate()
print(osemosys_results)

ModelResults (PyPSA)
Regions: ['REGION1']
Connections: 0
Technologies with capacity: 2
Objective: 200.4126

ModelResults (OSeMOSYS)
Regions: ['REGION1']
Connections: 0
Technologies with capacity: 2
Objective: 195.5729


In [19]:
# --- Cross-model comparison ---
tables = compare(pypsa_results, osemosys_results)

print("=== Topology Comparison ===")
print(tables["topology"])
print()
print("=== Supply Comparison (Installed Capacity) ===")
print(tables["supply"])
print()
print("=== Objective Comparison ===")
print(tables["objective"])

=== Topology Comparison ===
      NODE  IN_PyPSA  IN_OSeMOSYS  MATCH
0  REGION1      True         True   True

=== Supply Comparison (Installed Capacity) ===
    REGION   TECHNOLOGY  YEAR     PyPSA  OSeMOSYS          DIFF  MATCH
0  REGION1     GAS_CCGT  2026  0.012684  0.012684 -1.679351e-08   True
1  REGION1  GAS_TURBINE  2026  0.000000  0.000000  0.000000e+00   True

=== Objective Comparison ===
      model   objective
0     PyPSA  200.412554
1  OSeMOSYS  195.572923
2      diff    4.839631
